In [5]:
import re, sys, os
from pprint import pprint
from pathlib import Path
from operator import xor
from collections import defaultdict, Counter

patterns = [
    (b' %s ', bytes),
    (b' %s. ', bytes),
    (b' %s,', bytes),
    (b' %s.', bytes),
    (b' %s, ', bytes),
    (b'. %s ', bytes),
    (b'. %s ', bytes.title),
    (b', %s ', bytes),
    (b'.%s ', bytes),
    (b'.%s ', bytes.title),
    (b',%s ', bytes),
]

def test_rez(rez):
    ss = rez.split()  # .lower()
    for s in ss:
        m = re.match(rb'^([A-Z]?[a-z]+-?[a-z]*)|(\d+[-.,]?\d*)[.,]?$', s)
#        m = re.match(rb'^[-A-Za-z0-9.,]+$', s)
        if not m:
            return
    return True

In [6]:
def hexToBytes(s):
    bs = []
    for i in range(0, len(s), 2):
        scod = s[i: i + 2]
        bs.append(int(scod, 16))
    return  bytes(bs)

# Зашифрований текст, перетворений на байтові рядки
txt1 = Path("shifrovka.txt").read_text()
lines = [hexToBytes(s.strip()) for s in txt1.splitlines() if s.strip()]
lines = dict(enumerate(lines))
print(len(lines))

txt2 = Path("en_words.txt").read_text()
words = [bytes(s.strip(), encoding='utf8') for s in txt2.splitlines() if s.strip()]
words = sorted(words, key=len, reverse=True) 
print(len(words))

19
1013


In [10]:
# сортовані по спаду довжині рядка
lines2 = dict(enumerate(sorted(lines.items(), reverse=True, key=lambda x: len(x[1]))))
# несортовані, порядок як у початковій шифровці
# 
lines2 = dict(enumerate(lines.items()))
# Якщо кожний рядок зашифрований спільним ключем (послідовністю байтів),
# спробуємо знайти можливі значення і їх статистику для кожного байту
rez_codes = {}
for word0 in words:
    for ps in patterns:
        word = ps[1](ps[0] % word0)
        # print(word)
        # зсувом слова word для XOR кожної пари рядків шукаємо правдоподібний текст
        # якщо він правдоподібний для попередніх і наступних рядків, то скоріш за все
        # слово правильне і треба добавити статистику для цих позицій
        # (можна не тільки сусідніх, а в будьяких, тільки це накладніше)
        for L in range(1, len(lines2) - 1):
            # результат побайтного XOR для поточного і поаереднього рядків
            xorL0 = bytes(map(xor, lines2[L][1], lines2[L-1][1]))
            # результат побайтного XOR для поточного і наступного рядків
            xorL2 = bytes(map(xor, lines2[L][1], lines2[L+1][1]))
            # готовимо позицію зсувів
            for i in range(min(len(xorL0), len(xorL2)) - len(word) + 1):
                # одержуемо результати з зсунутими word-ами
                rez0 = bytes(map(xor, word, xorL0[i:]))
                rez2 = bytes(map(xor, word, xorL2[i:]))
                # перевіряємо правдоподібність результатів
                if test_rez(rez0) and test_rez(rez2):
                    # результати правдоподібні, добавляемо статистику
                    icodes = bytes(map(xor, word, lines2[L][1][i:]))
                    for j, cod in enumerate(icodes):
                        rez_codes.setdefault(i + j, Counter())[cod] += 1
rez_codes = {n: kvs.most_common(1)[0][0] for n, kvs in rez_codes.items()}

#print("'''")
print(''.join(str(rez_codes.get(i, 500) // 100 or ' ') for i in range(50)))
print(''.join(str(rez_codes.get(i, 500) % 100 // 10) for i in range(50)))
print(''.join(str(rez_codes.get(i, 500) % 10) for i in range(50)))
print(''.join(str(i // 10 or ' ') for i in range(50)))
print(''.join(str(i % 10) for i in range(50)))
for line in lines.values():
# for line in lines:
    rez = []
    for i, b in enumerate(line):
        ch = ord('_')
        if i in rez_codes:
            ch = b ^ rez_codes[i]
        rez.append(ch)
    print(bytes(rez).decode(encoding='utf8', errors='replace'))
print("'''")
# print(dict(sorted(rez_codes.items())))
print('rez_codes = {\n' + ''.join(f'{i:2}:{rez_codes.get(i, "00"):3}, ' + 
      '\n'*(i % 10 == 9)  for i in range(50)) + "}")

  11   2111211  1 1 111112222221  1 1     22155555
24890962157437431481683112513445741462848722300000
62768117232883984863594797464029210689294097300000
          1111111111222222222233333333334444444444
01234567890123456789012345678901234567890123456789
2'r who wotld bear tde whips and sch!n0o!2u9,___
  'oppressnr's wrong  the proud man          ___
  e pangs nf dispriz+d love, the laptsc++844i
  e insoleoce of offece, and the spr!n0
  at patieot merit oj th'unworthy tf8e0c
# en he hilself mighx his quietus mf8e
#!th a bard bodkin? [ho would fardek  !*/&y
 ' grunt aod sweat ubder a weary lia6,
6=t that tie dread oj something aftb! '*/ =a
  e undiscnvere'd coyntry, from whot6 ! ;&;
:' travelldr returns  puzzles the wn?lo
5&d makes ts rather near those ills'$ec'/"0
  an fly tn others tdat we know not'<f|
  us conschence doth,make cowards oasu0o/89a
5&d thus tie native due of resolutih=
=; sickliee o'er witd the pale cast'<fc;&; *-__
5&d enterpsises of g~eat pith and mh>e-;
#!th this seg

In [9]:
''' rez_codes = результат для  сортованих по спаду довжини рядків
5511   2111211  1 1 111112222221  1 2  1  21112555
00890962157437431481683112513445741452129956361000
00768217233883984863594797464029210119761422304000
          1111111111222222222233333333334444444444
01234567890123456789012345678901234567890123456789
??r woo would bear tde whips and scorns .*lx,!<?
??'opwressor's wrong  the proud man's co/89a   ?
??e pfngs of dispriz+d love, the law's d$ -ui
??e iisolence of offece, and the spurns
??at watient merit oj th'unworthy takes,
??en oe himself mighx his quietus make
??th f bare bodkin? [ho would fardels be >`
?? grrnt and sweat ubder a weary life,
??t toat the dread oj something after de 8$ 
??e uidiscovere'd coyntry, from whose bo4>"
?? trfveller returns  puzzles the will,
??d mfkes us rather near those ills we h :)
??an aly to others tdat we know not of?
??us donscience doth,make cowards of us     
??d tous the native due of resolution
?? sidklied o'er witd the pale cast of t)#9k-8u
??d eiterprises of g~eat pith and moment
??th shis regard theer currents turn awr8
??d lhse the name of,action.
'''
rez_codes = {  # результат для  сортованих по спаду довжини рядків
 0:   ,  1:   ,  2:187,  3:196,  4:  8,  5: 92,  6: 61,  7:227,  8:112,  9:153, 
10:173, 11:248, 12:138, 13:173, 14: 49, 15: 38, 16:114, 17: 48, 18:186, 19: 13, 
20:165, 21:189, 22:134, 23:117, 24:119, 25:227, 26:254, 27:216, 28:234, 29:240, 
30:242, 31:159, 32: 72, 33: 41, 34:110, 35: 41, 36:251, 37: 29, 38: 17, 39:126, 
40: 91, 41: 94, 42:252, 43:162, 44:133, 45:160, 46:214, 47: 00, 48: 00, 49: 00, 
}
rez_codes.update({5: 91, 10:172, 35: 46, 36:168, 38: 82, 39: 49, 40: 84, 41: 70, 42:229, 43:227, })

SyntaxError: invalid syntax (Temp/ipykernel_29984/2097584891.py, line 28)

In [ ]:
rez_codes = {  # результат для несортованих рядків, порядок як у початковій шифровці
 0: 26,  1: 42,  2:187,  3:196,  4:  8,  5: 91,  6: 61,  7:227,  8:112,  9:153, 
10:172, 11:248, 12:138, 13:173, 14: 49, 15: 38, 16:114, 17: 48, 18:186, 19: 13, 
20:165, 21:189, 22:134, 23:117, 24:119, 25:227, 26:254, 27:216, 28:234, 29:240, 
30:242, 31:159, 32: 72, 33: 41, 34:110, 35: 46, 36:168, 37: 29, 38: 82, 39: 49, 
40: 84, 41: 70, 42:229, 43:227, 44:133, 45: 00, 46: 00, 47: 00, 48: 00, 49: 00, 
}
''' rez_codes = результат для несортованих рядків, порядок як у початковій шифровці
  11   2111211  1 1 111112222221  1 1     22155555
24890962157437431481683112513445741462848722300000
62768117232883984863594797464029210689294097300000
          1111111111222222222233333333334444444444
01234567890123456789012345678901234567890123456789
2'r who wotld bear tde whips and sch!n0o!2u9,???
  'oppressnr's wrong  the proud man          ???
  e pangs nf dispriz+d love, the laptsc++844i
  e insoleoce of offece, and the spr!n0
  at patieot merit oj th'unworthy tf8e0c
# en he hilself mighx his quietus mf8e
#!th a bard bodkin? [ho would fardek  !*/&y
 ' grunt aod sweat ubder a weary lia6,
6=t that tie dread oj something aftb! '*/ =a
  e undiscnvere'd coyntry, from whot6 ! ;&;
:' travelldr returns  puzzles the wn?lo
5&d makes ts rather near those ills'$ec'/"0
  an fly tn others tdat we know not'<f|
  us conschence doth,make cowards oasu0o/89a
5&d thus tie native due of resolutih=
=; sickliee o'er witd the pale cast'<fc;&; *-??
5&d enterpsises of g~eat pith and mh>e-;
#!th this segard theer currents turisa4=7
5&d lose tie name of,action.
'''

In [11]:
from copy import deepcopy
from functools import reduce 

rez2_codes = deepcopy(rez_codes)
rez2_codes = {  # результат для  сортованих по спаду довжини рядків
 0: 00,  1: 00,  2:187,  3:196,  4:  8,  5: 92,  6: 61,  7:227,  8:112,  9:153, 
10:173, 11:248, 12:138, 13:173, 14: 49, 15: 38, 16:114, 17: 48, 18:186, 19: 13, 
20:165, 21:189, 22:134, 23:117, 24:119, 25:227, 26:254, 27:216, 28:234, 29:240, 
30:242, 31:159, 32: 72, 33: 41, 34:110, 35: 41, 36:251, 37: 29, 38: 17, 39:126, 
40: 91, 41: 94, 42:252, 43:162, 44:133, 45:160, 46:214, 47: 00, 48: 00, 49: 00, 
}
rez2_codes.update({
# доповнення до несортованих рядків, порядок як у початковій шифровці
  0: 26,  
  1: 42, 
#  5: 91, 
#  10:172, 
#  35: 46, 
#  36:168, 
#  38: 82, 
#  39: 49, 
#  40: 84, 
#  41: 70, 
#  42:229, 
#  43:227, 
})
rez2_codes.update({
# ручне доповнення
})

# розрахункове доповнення
mauals = {
  5: b'oh', 
 20: b"dh", 
 0: b'5a', 
 1: b'&n', 
 40: b'n/', 
 41: b't8', 
 42: b'u9', 
 43: b'ma', 
 44: b'e ', 
 45: b'l ', 
 46: b'y ', 
 47: b',k', 
#  3: b' x,t', 
#  10: b'no',
#  20: b'hd',
}
for p, x in mauals.items():
    rez2_codes[p] ^= reduce(xor, x, 0)

#print("'''")
print(''.join(str(rez2_codes.get(i, 500) // 100 or ' ') for i in range(50)))
print(''.join(str(rez2_codes.get(i, 500) % 100 // 10) for i in range(50)))
print(''.join(str(rez2_codes.get(i, 500) % 10) for i in range(50)))
print(''.join(str(i // 10 or ' ') for i in range(50)))
print(''.join(str(i % 10) for i in range(50)))
for line in lines.values():
# for line in lines:
    rez = []
    for i, b in enumerate(line):
        ch = ord('?')
        if i in rez2_codes:
            ch = b ^ rez2_codes[i]
        rez.append(ch)
    print(bytes(rez).decode(encoding='utf8', errors='replace'))
print("'''")
# print(dict(sorted(rez2_codes.items())))
print('rez2_codes = {\n' + ''.join(f'{i:2}:{rez2_codes.get(i, "00"):3}, ' + 
      '\n'*(i % 10 == 9)  for i in range(50)) + "}")

  11   2111211  1 1 111112222221  1 2  1  11121   
79890962157437431481683112513445741452122177934700
88768117233883984863994797464029210119766864263100
          1111111111222222222233333333334444444444
01234567890123456789012345678901234567890123456789
for who would bear the whips and scorns of time,
th'oppressor's wrong, the proud man's contumely,
the pangs of dispriz'd love, the law's delay,
the insolence of office, and the spurns
that patient merit of th'unworthy takes,
when he himself might his quietus make
with a bare bodkin? Who would fardels bear,
to grunt and sweat under a weary life,
but that the dread of something after death,
the undiscovere'd country, from whose bourn
no traveller returns, puzzles the will,
and makes us rather bear those ills we have
than fly to others that we know not of?
thus conscience doth make cowards of us all,
and thus the native hue of resolution
is sicklied o'er with the pale cast of thought,
and enterprises of great pith and moment
with this reg

In [ ]:
rez2_codes = {
 0: 26,  1: 42,  2:187,  3:196,  4:  8,  5: 91,  6: 61,  7:227,  8:112,  9:153, 
10:172, 11:248, 12:138, 13:173, 14: 49, 15: 38, 16:114, 17: 48, 18:186, 19: 13, 
20:165, 21:189, 22:134, 23:117, 24:119, 25:227, 26:254, 27:216, 28:234, 29:240, 
30:242, 31:159, 32: 72, 33: 41, 34:110, 35: 46, 36:168, 37: 29, 38: 82, 39: 49, 
40: 84, 41: 70, 42:229, 43:227, 44:133, 45:   , 46:   , 47:   , 48:   , 49:   , 
}
'''
  11   2111211  1 1 111112222221  1 1     22155555
24890962157437431481683112513445741462848722300000
62768117232883984863594797464029210689294097300000
          1111111111222222222233333333334444444444
01234567890123456789012345678901234567890123456789
2'r who wotld bear tde whips and sch!n0o!2u9,???
  'oppressnr's wrong  the proud man          ???
  e pangs nf dispriz+d love, the laptsc++844i
  e insoleoce of offece, and the spr!n0
  at patieot merit oj th'unworthy tf8e0c
# en he hilself mighx his quietus mf8e
#!th a bard bodkin? [ho would fardek  !*/&y
 ' grunt aod sweat ubder a weary lia6,
6=t that tie dread oj something aftb! '*/ =a
  e undiscnvere'd coyntry, from whot6 ! ;&;
:' travelldr returns  puzzles the wn?lo
5&d makes ts rather near those ills'$ec'/"0
  an fly tn others tdat we know not'<f|
  us conschence doth,make cowards oasu0o/89a
5&d thus tie native due of resolutih=
=; sickliee o'er witd the pale cast'<fc;&; *-??
5&d enterpsises of g~eat pith and mh>e-;
#!th this segard theer currents turisa4=7
5&d lose tie name of,action.
'''


In [ ]:
'''
  11   2111211  1 1 111112222221  1 2  1  11121   
79890962157437431481683112513445741452122177934700
88768117233883984863994797464029210119766864263100
          1111111111222222222233333333334444444444
01234567890123456789012345678901234567890123456789
for who would bear the whips and scorns of time,
th'oppressor's wrong, the proud man's contumely,
the pangs of dispriz'd love, the law's delay,
the insolence of office, and the spurns
that patient merit of th'unworthy takes,
when he himself might his quietus make
with a bare bodkin? Who would fardels bear,
to grunt and sweat under a weary life,
but that the dread of something after death,
the undiscovere'd country, from whose bourn
no traveller returns, puzzles the will,
and makes us rather bear those ills we have
than fly to others that we know not of?
thus conscience doth make cowards of us all,
and thus the native hue of resolution
is sicklied o'er with the pale cast of thought,
and enterprises of great pith and moment
with this regard their currents turn awry
and lose the name of action.
'''
rez2_codes = {
 0: 78,  1: 98,  2:187,  3:196,  4:  8,  5: 91,  6: 61,  7:227,  8:112,  9:153, 
10:173, 11:248, 12:138, 13:173, 14: 49, 15: 38, 16:114, 17: 48, 18:186, 19: 13, 
20:169, 21:189, 22:134, 23:117, 24:119, 25:227, 26:254, 27:216, 28:234, 29:240, 
30:242, 31:159, 32: 72, 33: 41, 34:110, 35: 41, 36:251, 37: 29, 38: 17, 39:126, 
40: 26, 41: 18, 42:176, 43:174, 44:192, 45:236, 46:143, 47: 71, 48:  0, 49:  0, 
}
